# IMDB Sample

The following example demonstrates how to use Wallaroo with chained models.  In this example, we will be using information from the IMDB (Internet Movie DataBase) with a sentiment model to detect whether a given review is positive or negative.  Imagine using this to automatically scan Tweets regarding your product and finding either customers who need help or have nice things to say about your product.

Note that this example is considered a "toy" model - only the first 100 words in the review were tokenized, and the embedding is very small.

The following example is based on the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/), and sample data can be downloaded from the [aclIMDB dataset](http://s3.amazonaws.com/text-datasets/aclImdb.zip ).

## Open a Connection to Wallaroo

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

In [1]:
import wallaroo

In [ ]:
wl = wallaroo.Client()

To test this model, we will perform the following:

* Create a workspace for our models.
* Upload two models:
  * `embedder`: Takes pre-tokenized text documents (model input: 100 integers/datum; output 800 numbers/datum) and creates an embedding from them.
  * `sentiment`:  The second model classifies the resulting embeddings from 0 to 1, which 0 being an unfavorable review, 1 being a favorable review.
* Create a pipeline that will take incoming data and pass it to the embedder, which will pass the output to the sentiment model, and then export the final result.
* To test it, we will use information that has already been tokenized and submit it to our pipeline and gauge the results.

First we'll create a workspace for our environment, and call it `imdb-workspace`:

In [3]:
new_workspace = wl.create_workspace("imdb-workspace-test")
_ = wl.set_current_workspace(new_workspace)

Just to make sure, let's list our current workspace.  If everything is going right, it will show us we're in the `imdb-workspace`.

In [4]:
wl.get_current_workspace()

{'name': 'imdb-workspace-test', 'id': 11, 'archived': False, 'created_by': '5378aa8b-aa65-4b17-91ab-9251a77523c4', 'created_at': '2022-07-20T21:18:39.149877+00:00', 'models': [], 'pipelines': []}

Now we'll upload our two models:

* `embedder.onnx`: This will be used to embed the tokenized documents for evaluation.
* `sentiment_model.onnx`: This will be used to analyze the review and determine if it is a positive or negative review.  The closer to 0, the more likely it is a negative review, while the closer to 1 the more likely it is to be a positive review.

In [5]:
embedder = wl.upload_model('embedder-o', './embedder.onnx').configure()
smodel = wl.upload_model('smodel-o', './sentiment_model.onnx').configure(runtime="onnx", tensor_fields=["flatten_1"])

With our models uploaded, now we'll create our pipeline that will contain two steps:

* First, it runs the data through the embedder.
* Second, it applies it to our sentiment model.

In [6]:
# now make a pipeline
imdb_pipeline = (wl.build_pipeline("imdb-pipeline-sample")
                        .add_model_step(embedder)
                        .add_model_step(smodel))

Now that we have our pipeline set up with the steps, we can deploy the pipeline.

In [7]:
imdb_pipeline.deploy()

Waiting for deployment - this will take up to 45s ..... ok


name,imdb-pipeline-sample
created,2022-07-20 21:18:40.580032+00:00
last_updated,2022-07-20 21:18:40.647792+00:00
deployed,True
tags,
steps,embedder-o


We'll check the pipeline status to verify it's deployed and the models are ready.

In [8]:
imdb_pipeline.status()

{'status': 'Running',
 'details': None,
 'engines': [{'ip': '10.48.0.105',
   'name': 'engine-76d8c58549-hpfzr',
   'status': 'Running',
   'reason': None,
   'pipeline_statuses': {'pipelines': [{'id': 'imdb-pipeline-sample',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'smodel-o',
      'version': '146bfce2-9771-44b7-a818-ee2f7da541d2',
      'sha': '3473ea8700fbf1a1a8bfb112554a0dde8aab36758030dcde94a9357a83fd5650',
      'status': 'Running'},
     {'name': 'embedder-o',
      'version': '2b7bf983-a128-4116-8afe-665d640f29c7',
      'sha': 'd083fd87fa84451904f71ab8b9adfa88580beb92ca77c046800f79780a20b7e4',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.48.0.104',
   'name': 'engine-lb-85846c64f8-bvzc7',
   'status': 'Running',
   'reason': None}]}

To test this out, we'll start with a single piece of information from our data directory.

In [10]:
results = imdb_pipeline.infer_from_file('./data/singleton.json')

results[0].data()

[array([[0.37142318]])]

Since that works, let's load up all 50 rows and do a full inference on each of them.  Note that Jupyter Hub has a size limitation, so for production systems the outputs should be piped out to a different output.

In [11]:
# for the victory lap, infer on all 50 rows
results = imdb_pipeline.infer_from_file('./data/test_data.json')
results[0].data()

[array([[3.71423185e-01],
        [9.65576112e-01],
        [7.60161877e-02],
        [2.46452361e-01],
        [8.63283277e-02],
        [6.39613509e-01],
        [2.47336328e-02],
        [5.02990067e-01],
        [9.34223831e-01],
        [7.17751265e-01],
        [2.04768777e-03],
        [3.55861127e-01],
        [2.48722464e-01],
        [2.73299277e-01],
        [9.60162282e-03],
        [4.95020479e-01],
        [8.30442309e-02],
        [5.34835458e-02],
        [2.74230242e-02],
        [1.26478374e-02],
        [2.39091218e-02],
        [8.63728166e-01],
        [1.57089770e-01],
        [3.46490622e-01],
        [3.56459022e-01],
        [7.97988474e-02],
        [6.78595304e-02],
        [3.17764282e-03],
        [4.39540178e-01],
        [3.33117247e-02],
        [1.46508217e-04],
        [7.39861846e-01],
        [1.51472032e-01],
        [2.41219997e-04],
        [2.69098580e-02],
        [9.06612277e-01],
        [8.55922699e-04],
        [4.60651517e-03],
        [4.5

## Undeploy

With our pipeline's work done, we'll undeploy it and give our Kubernetes environment back its resources.

In [12]:
imdb_pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ............................... ok


name,imdb-pipeline-sample
created,2022-07-20 21:18:40.580032+00:00
last_updated,2022-07-20 21:18:40.647792+00:00
deployed,False
tags,
steps,embedder-o


And there is our example. Please feel free to contact us at Wallaroo for if you have any questions.